# C3 Project 1: Image Classification

In [3]:
from main import test, train, TrainDataset, TestDataset
from bovw import BOVW
# metrics 
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay,
)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Charge the data

In [4]:
data_train = TrainDataset()
data_test = TestDataset()

### Experiment 1

In [ ]:
# HYPERPARAMETERS
detector_type = "DenseSIFT"
codebook_size = 500
classifier_algorithm = "SVM"
detector_kwargs = {}
stride=8
scale=2

# Compute cache paths
kwarg_detector_str = [f"_{str(key)}-{str(value)}" for key, value in detector_kwargs.items()]
kwarg_detector_str = "".join(kwarg_detector_str)
cache_file_train = "./cache_train/"+detector_type+kwarg_detector_str+".pkl"
cache_file_test = "./cache_test/"+detector_type+kwarg_detector_str+".pkl"

print(cache_file_test)

./cache_test/DenseSIFT.pkl


## Run experiment

In [12]:
bovw = BOVW(detector_type=detector_type, codebook_size=codebook_size, detector_kwargs=detector_kwargs, stride=stride, scale=scale)

if classifier_algorithm == 'LogisticRegression':
        classifier = LogisticRegression(class_weight="balanced")
elif classifier_algorithm == 'SVM':
        classifier = SVC(class_weight="balanced", probability=True, random_state=42, C=10)

# TRAIN
y_pred_train, y_probas_train, labels_train = train(dataset=data_train, bovw=bovw, classifier=classifier, cache_file=cache_file_train)
train_acc = accuracy_score(y_true=labels_train, y_pred=y_pred_train)
print("Accuracy on Phase[Train]:", train_acc)

# TEST
y_pred_test, y_probas_test, labels_test = test(dataset=data_test, bovw=bovw, classifier=classifier, cache_file=cache_file_test)
test_acc = accuracy_score(y_true=labels_test, y_pred=y_pred_test)
print("Accuracy on Phase[Test]:", test_acc)

Phase[Train]: Loading descriptors from ./cache_train/DenseSIFT.pkl
Fitting the codebook (took 36.58 seconds)
Computing the bovw histograms
Fitting the classifier
Classifier fitted.
Accuracy on Phase[Train]: 0.34160919540229884
Phase[Test]: Loading descriptors from ./cache_test/DenseSIFT.pkl
Computing the bovw histograms
Predicting the values
Accuracy on Phase[Test]: 0.3490909090909091
